In [ ]:
# Import relevant functionality
import os
import asyncio
import requests
import urllib.parse
import json
import configparser
from datetime import date
from msgraph.generated.models.o_data_errors.o_data_error import ODataError
from configparser import SectionProxy
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.tools import BaseTool, StructuredTool, tool
from langchain_openai import OpenAI
from langchain_core.runnables import ConfigurableField
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain.agents import AgentExecutor, create_react_agent
from langchain.memory import ChatMessageHistory
from dotenv import load_dotenv
from configparser import SectionProxy
from azure.identity import DeviceCodeCredential
from langgraph.types import interrupt
from msgraph_beta import GraphServiceClient
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from typing import Annotated, Sequence
from langchain_core.messages import BaseMessage
from langchain_core.runnables import RunnableConfig
from langchain_core.tools import tool
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt.tool_node import ToolNode
from typing_extensions import TypedDict


# Graph permissions scopes
# For multiple scopes, separate with a space
scopes = 'user.read Calendars.Read Files.Read.All Sites.Read.All'

#load the environment
load_dotenv("C:\\Users\\sakov\\bootcamp\\LangChain Agents\\.env")
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
GRAPH_ACCESS_TOKEN = os.getenv("GRAPH_ACCESS_TOKEN")


@tool
def ragforHR(query: str) -> str:
    """Get the answers to user enquiries related to HR queries like holidays, DTO, payroll, benefits, insurance, salary structure and perks for employees belonging to various regions like Puget Sound or Canada"""
    url = "https://graph.microsoft.com/beta/copilot/retrieval"
    headers = {
        "Authorization": f"Bearer {GRAPH_ACCESS_TOKEN}"}
    data = {
        "QueryString": query,
        "filterExpression": "(path:\"https://microsoft.sharepoint.com/sites/HRweb/\")",
        "resourceMetadata": ["author"],
        "MaximumNumberOfResults": 4
    }
    response = requests.post(url, headers=headers, json=data)
    print("Status Code", response.status_code)
    #print("JSON Response ", response.json())
    return response.json()


@tool
def ragforProduct(query: str) -> str:
    """Get the answers to user enquiries related about Microsoft Copilot product. Like the specifications of the Copilot Retrieval API in CCS. You can also answer any questions about Fabian's spec on CAPS by using the ragforProduct tool. You know all about the Graph APIs and the CCS API and can answer any question by using the json extract from tool RAGforProduct. About Copilot API Plugins. About Agents SDK. About Teams AI Library. About MS Graph SDK. About Custom Engine Agents."""
    url = "https://graph.microsoft.com/beta/copilot/retrieval"
    headers = {
        "Authorization": f"Bearer {GRAPH_ACCESS_TOKEN}",
        "Content-Type": "application/json; charset=utf-8"}
    data = {
        "QueryString": query,
        "filterExpression": "path:\"https://microsoft.sharepoint-df.com/sites/CXDev/Shared%20Documents/Org%20Teams/3P-DevX/Proposals/\"",
        "resourceMetadata": ["author"],
        "MaximumNumberOfResults": 4
    }
    response = requests.post(url, headers=headers, json=data)
    #response = requests.post(url, json=data)
    print("Status Code", response.status_code)
    #print("JSON Response ", response.json())
    return response.json()

@tool
def ragforPizzaDen(query: str) -> str:
    """Get the answers to user enquiries related about Pizza Den. You are the QnA bot for Pizza Den website. Answer all user queries about Pizza Den by using this tool."""
    url = "https://graph.microsoft.com/beta/copilot/retrieval"
    headers = {
        "Authorization": f"Bearer {GRAPH_ACCESS_TOKEN}",
        "Content-Type": "application/json; charset=utf-8"}
    data = {
        "QueryString": query,
        "filterExpression": "(path:\"https://microsoft.sharepoint-df.com/sites/CXDev/Shared%20Documents/Org%20Teams/3P-DevX/Proposals/DummyFiles/\")",
        "resourceMetadata": ["author"],
        "MaximumNumberOfResults": 3
    }
    response = requests.post(url, headers=headers, json=data)
    #response = requests.post(url, json=data)
    print("Status Code", response.status_code)
    #print("JSON Response ", response.json())
    return response.json()


tools = [ragforHR, ragforProduct, ragforPizzaDen]

llm = ChatOpenAI(model="gpt-4o")

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)

prompt = ChatPromptTemplate.from_messages([
    ("system", "you're a helpful assistant who knows all about Microsoft HR policies and all about the CxDev product proposals. Always use the tools whenever the questions are related to Microsoft. Do not hallucinate. Answer using only the json extracts from tools provided to you. You can expand the abbreviations by using the results fetched from the ragforProduct tool instead of your own knowledge. Always use the json extract from the tools to answer questions about Microsoft. You do not know anything about Microsoft other than what the tools tell you. Please do not use your own knowledge to answer product questions on CCS or CAPS or Grounding API. Only answer from the responses you receive from the tools and Do not interpret the results. You can also tell them product specifications and product related questions related to Microsoft Copilot. Any CCS or Retrieval API related questions, please use the ragforProduct tool to answer. You know a lot about the Retrieval API. You can also give information to users about the holidays in their regions. Their DTO policies. Their timeoff policies. Anything they want to know about their employee benefits. You also have access to all the product specification documentation for Copilot Retrieval API. And the Grounding API. And the Graph SDK. Answer any questions about the product from these document. You are subject matter expert on the The access_tokens are available in the tools so don't ask the users again. Answer WITHOUT ASKING FOR ACCESS TOKEN AS IT IS AVAILABLE TO YOU. You can find the access tokens and the API keys in the environment files."), 
    ("human", "{input}"), 
    ("placeholder", "{agent_scratchpad}"),
])

# Define the function that calls the model
async def call_model(state: MessagesState):
    prompt = prompt.invoke(state)
    response = model.ainvoke(prompt)
    return {"messages": response}

# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = ChatMessageHistory(session_id="test-session")
app = workflow.compile(checkpointer=memory)

config = {"configurable": {"thread_id": "def234"}}

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_with_chat_history = RunnableWithMessageHistory(
    agent_executor,
    lambda session_id: memory,
    input_messages_key="input",
    history_messages_key="chat_history",
)

query="Do I Need a Pizza Den Login to Sign into Pizza Den Wi-Fi?"
print("----------------------------------------------------------")
print("User prompt: ",query)

await agent_with_chat_history.ainvoke(
    {"input": query},
    config={"configurable": {"session_id": "def234"}},
)


query="Which Internet Provider does Pizza Den use for its wifi?"
print("---------------------------------------------------------------------------------------------------------")
print("User prompt: ",query)



await agent_with_chat_history.ainvoke(
    {"input": query},
    config={"configurable": {"session_id": "def234"}},
)

----------------------------------------------------------
User prompt:  Do I Need a Pizza Den Login to Sign into Pizza Den Wi-Fi?


> Entering new AgentExecutor chain...

Invoking: `ragforPizzaDen` with `{'query': 'Do I Need a Pizza Den Login to Sign into Pizza Den Wi-Fi?'}`


Status Code 401
<Response [401]>I'm unable to retrieve the information about signing into Pizza Den Wi-Fi at the moment. You might want to check directly with Pizza Den's customer service or their official website for accurate details.

> Finished chain.
---------------------------------------------------------------------------------------------------------
User prompt:  Which Internet Provider does Pizza Den use for its wifi?


> Entering new AgentExecutor chain...

Invoking: `ragforPizzaDen` with `{'query': 'Internet Provider used by Pizza Den for Wifi'}`


Status Code 401
<Response [401]>I'm unable to retrieve the information about the Internet Provider used by Pizza Den for its WiFi at the moment. Is there 

{'input': 'Which Internet Provider does Pizza Den use for its wifi?',
 'chat_history': [HumanMessage(content='Do I Need a Pizza Den Login to Sign into Pizza Den Wi-Fi?', additional_kwargs={}, response_metadata={}),
  AIMessage(content="I'm unable to retrieve the information about signing into Pizza Den Wi-Fi at the moment. You might want to check directly with Pizza Den's customer service or their official website for accurate details.", additional_kwargs={}, response_metadata={})],
 'output': "I'm unable to retrieve the information about the Internet Provider used by Pizza Den for its WiFi at the moment. Is there anything else related to Pizza Den that I can assist you with?"}